In [6]:
#User Input
lib_type = "single_end"
seq_platform = "illumina"
read_length = 100

In [5]:
source("https://bioconductor.org/biocLite.R")
biocLite("dada2")
biocLite("phyloseq")
biocLite("biom")

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.4 (2018-03-15).
Installing package(s) ‘dada2’
also installing the dependencies ‘formatR’, ‘matrixStats’, ‘colorspace’, ‘assertthat’, ‘utf8’, ‘lambda.r’, ‘futile.options’, ‘DelayedArray’, ‘RColorBrewer’, ‘dichromat’, ‘munsell’, ‘labeling’, ‘cli’, ‘pillar’, ‘futile.logger’, ‘snow’, ‘bitops’, ‘SummarizedExperiment’, ‘RCurl’, ‘GenomeInfoDbData’, ‘BiocGenerics’, ‘S4Vectors’, ‘IRanges’, ‘XVector’, ‘gtable’, ‘lazyeval’, ‘plyr’, ‘rlang’, ‘scales’, ‘tibble’, ‘viridisLite’, ‘BiocParallel’, ‘Rsamtools’, ‘GenomicAlignments’, ‘Biobase’, ‘GenomeInfoDb’, ‘GenomicRanges’, ‘hwriter’, ‘zlibbioc’, ‘latticeExtra’, ‘BH’, ‘Biostrings’, ‘ggplot2’, ‘data.table’, ‘reshape2’, ‘ShortRead’, ‘RcppParallel’

installation path not wri

In [ ]:
library("dada2")
library("phylogen")
library("biom")

In [3]:
#Read in files
path = "./"
filtpath = file.path(path, "filtered")
input_files = list.files(path, pattern="fastq.gz")

ERROR: Error in library("biom"): there is no package called ‘biom’


In [ ]:
#Plot quality profiles of files
plotQualityProfile(input_files)

In [ ]:
#Filter and Trim the reads
filterAndTrim(file.path(path, input_files), file.path(filtpath, input_files), compress=TRUE, verbose=TRUE, multithread=TRUE)

In [ ]:
#Parse filtered files to learn error rates, infer sequence variants and construct a seq-table
filtered_files = list.files(filtpath, pattern="fastq.gz", full.names=TRUE)
samples = sapply(basename(filtered_files))
names(filtered_files) = samples

error_rates = learnErrors(filtered_files, nbases=1e8, multithread=TRUE, randomize=TRUE)
plotErrors(error_rates, nominalQ=TRUE)


In [ ]:
seq_variants = vector("list", length(samples))
names(seq_variants) = samples

for i in samples:
    dereplicated = derepFastq(filtered_files[[i]])
    seq_variants[[i]] = dada(dereplicated, err=error_rates, multithread=TRUE)
    
sequence_table = makeSequenceTable(seq_variants)

In [ ]:
#Remove chimeras and assign taxonomy

sequence_table = removeBimeraDenovo(sequence_table, method="consensus", multithread=TRUE)
taxa_assign = assignTaxonomy(sequence_table, "gg_13_8_train_set_97.fa.gz", multithread=TRUE)



In [ ]:
#Get OTU Table

otu_table = otu_table(sequence_table, taxa_are_rows=FALSE)
